In [46]:
import pandas as pd
import numpy as np
import getpass
import snowflake.connector
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go


In [47]:

# Configuración global
stocks = ['BBVA.MC', 'IBE.MC', 'ITX.MC', 'REP.MC']  # Tickers IBEX
start_date = '2020-01-01'
end_date = '2024-12-31'
NUM_PORTFOLIOS = 10000
NUM_TRADING_DAYS = 252
RISK_FREE_RATE = 0.02 #! REVISA JROMERO

In [51]:
# Solicitar contraseña y conectar a Snowflake
password = input("🔓 Introduce tu contraseña de Snowflake (visible): ")
conn = snowflake.connector.connect(
    user='TFMGRUPO4',
    password=password,
    account='WYNIFVB-YE01854',
    warehouse='COMPUTE_WH',
    database='YAHOO_FINANCE',
    schema='MACHINE_LEARNING',
    role='ACCOUNTADMIN'
)


DatabaseError: 250001 (08001): Failed to connect to DB: WYNIFVB-YE01854.snowflakecomputing.com:443. Your user account has been temporarily locked. Try again later or contact your account administrator for assistance. For more information about this error, go to https://community.snowflake.com/s/error-your-user-login-has-been-locked.

In [ ]:

# Construir y ejecutar la consulta
cursor = conn.cursor()
tickers_str = ', '.join(f"'{ticker}'" for ticker in stocks)
query = f"""
    SELECT TICKER, FECHA, CLOSE
    FROM TICKERS_INDEX
    WHERE TICKER IN ({tickers_str})
      AND FECHA BETWEEN '{start_date}' AND '{end_date}'
"""
cursor.execute(query)

# Convertir a DataFrame
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[col[0] for col in cursor.description])
cursor.close()
conn.close()

# Pivotear y limpiar
df['FECHA'] = pd.to_datetime(df['FECHA'])
df_prices = df.pivot(index='FECHA', columns='TICKER', values='CLOSE')
df_prices = df_prices.sort_index().fillna(method='ffill').dropna()
